In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv

load_dotenv()

# create embedding model
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [3]:
from langchain_core.documents import Document

# Create LangChain documents for IPL players

doc1 = Document(
        page_content="Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.",
        metadata={"team": "Royal Challengers Bangalore"}
    )
doc2 = Document(
        page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
        metadata={"team": "Mumbai Indians"}
    )
doc3 = Document(
        page_content="MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.",
        metadata={"team": "Chennai Super Kings"}
    )
doc4 = Document(
        page_content="Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.",
        metadata={"team": "Mumbai Indians"}
    )
doc5 = Document(
        page_content="Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.",
        metadata={"team": "Chennai Super Kings"}
    )


In [4]:
docs = [doc1, doc2, doc3, doc4, doc5]

In [5]:
""" 
from langchain_community.vectorstores import Chroma

LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. 
An updated version of the class exists in the `langchain-chroma package and should be used instead. 
To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma`
"""

from langchain_chroma import Chroma

# Create vector store
vector_store = Chroma(
    embedding_function= embedding,
    persist_directory='chroma_db',
    collection_name='sample'
)

In [9]:
# Add document to store
vector_store.add_documents(docs)

['bdbca6e4-1c5b-403a-afd5-830d83a49630',
 '9ec6d434-598a-4cb1-82a3-6055ca3ec2e7',
 '77687531-246f-47d1-8597-4b556cde8362',
 '5b08e77a-c5db-4a99-882a-b28cfaf03619',
 'c50b04ef-6f6e-405b-85ab-2b6709605e35']

In [11]:
# Shows about documents
vector_store.get(include=['embeddings', 'documents', 'metadatas'])

{'ids': ['bdbca6e4-1c5b-403a-afd5-830d83a49630',
  '9ec6d434-598a-4cb1-82a3-6055ca3ec2e7',
  '77687531-246f-47d1-8597-4b556cde8362',
  '5b08e77a-c5db-4a99-882a-b28cfaf03619',
  'c50b04ef-6f6e-405b-85ab-2b6709605e35'],
 'embeddings': array([[ 0.00994724,  0.06914337, -0.05147111, ..., -0.03543338,
          0.01284805,  0.0124829 ],
        [ 0.00127743,  0.03129851, -0.02375379, ..., -0.00518362,
         -0.03280613,  0.02737714],
        [-0.10265914,  0.02650809,  0.02271499, ..., -0.03359744,
         -0.07984944, -0.01507706],
        [ 0.02123391, -0.02468549, -0.04494374, ..., -0.10995812,
          0.00572557,  0.09915379],
        [ 0.01873984,  0.04382846, -0.04304255, ..., -0.07801618,
         -0.0784068 , -0.00304188]], shape=(5, 384)),
 'documents': ['Virat Kohli is one of the most successful and consistent batsmen in IPL history. Known for his aggressive batting style and fitness, he has led the Royal Challengers Bangalore in multiple seasons.',
  "Rohit Sharma is the mo

In [15]:
# Perform similarity search 
vector_store.similarity_search(
    query='Who is the bowler among them?',
    k=1 # similarity vector
)

[Document(id='5b08e77a-c5db-4a99-882a-b28cfaf03619', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.')]

In [18]:
# With score
# Low score is better - it represent the distance
vector_store.similarity_search_with_score(
    query='Who is the bowler among them?',
    k=3 # No of similarity vector
)

[(Document(id='5b08e77a-c5db-4a99-882a-b28cfaf03619', metadata={'team': 'Mumbai Indians'}, page_content='Jasprit Bumrah is considered one of the best fast bowlers in T20 cricket. Playing for Mumbai Indians, he is known for his yorkers and death-over expertise.'),
  0.9801042675971985),
 (Document(id='9ec6d434-598a-4cb1-82a3-6055ca3ec2e7', metadata={'team': 'Mumbai Indians'}, page_content="Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure."),
  1.071707844734192),
 (Document(id='c50b04ef-6f6e-405b-85ab-2b6709605e35', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.154654622077942)]

In [ ]:
# Metadata filtering
vector_store.similarity_search_with_score(
    query='',
    filter={"team": "Chennai Super Kings"} # based on metadata
)

[(Document(id='77687531-246f-47d1-8597-4b556cde8362', metadata={'team': 'Chennai Super Kings'}, page_content='MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.'),
  1.8436006307601929),
 (Document(id='c50b04ef-6f6e-405b-85ab-2b6709605e35', metadata={'team': 'Chennai Super Kings'}, page_content='Ravindra Jadeja is a dynamic all-rounder who contributes with both bat and ball. Representing Chennai Super Kings, his quick fielding and match-winning performances make him a key player.'),
  1.890937089920044)]

In [20]:
# Update the document
updated_doc1 = Document(
    page_content="Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple centuries in a single season. Despite RCB not winning an IPL title under his captaincy, Kohli's passion and fitness set a benchmark for the league. His ability to chase targets and anchor innings has made him one of the most dependable players in T20 cricket.",
    metadata={"team": "Royal Challengers Bangalore"}
)

vector_store.update_document(
    document_id='bdbca6e4-1c5b-403a-afd5-830d83a49630',
    document= updated_doc1
)


In [21]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['bdbca6e4-1c5b-403a-afd5-830d83a49630',
  '9ec6d434-598a-4cb1-82a3-6055ca3ec2e7',
  '77687531-246f-47d1-8597-4b556cde8362',
  '5b08e77a-c5db-4a99-882a-b28cfaf03619',
  'c50b04ef-6f6e-405b-85ab-2b6709605e35'],
 'embeddings': array([[-0.00233745,  0.05902086, -0.04774048, ..., -0.07264046,
          0.00276781, -0.00344083],
        [ 0.00127743,  0.03129851, -0.02375379, ..., -0.00518362,
         -0.03280613,  0.02737714],
        [-0.10265914,  0.02650809,  0.02271499, ..., -0.03359744,
         -0.07984944, -0.01507706],
        [ 0.02123391, -0.02468549, -0.04494374, ..., -0.10995812,
          0.00572557,  0.09915379],
        [ 0.01873984,  0.04382846, -0.04304255, ..., -0.07801618,
         -0.0784068 , -0.00304188]], shape=(5, 384)),
 'documents': ["Virat Kohli, the former captain of Royal Challengers Bangalore (RCB), is renowned for his aggressive leadership and consistent batting performances. He holds the record for the most runs in IPL history, including multiple ce

In [24]:
# delete document
vector_store.delete(ids=['bdbca6e4-1c5b-403a-afd5-830d83a49630'])

In [25]:
# view documents
vector_store.get(include=['embeddings','documents', 'metadatas'])

{'ids': ['9ec6d434-598a-4cb1-82a3-6055ca3ec2e7',
  '77687531-246f-47d1-8597-4b556cde8362',
  '5b08e77a-c5db-4a99-882a-b28cfaf03619',
  'c50b04ef-6f6e-405b-85ab-2b6709605e35'],
 'embeddings': array([[ 0.00127743,  0.03129851, -0.02375379, ..., -0.00518362,
         -0.03280613,  0.02737714],
        [-0.10265914,  0.02650809,  0.02271499, ..., -0.03359744,
         -0.07984944, -0.01507706],
        [ 0.02123391, -0.02468549, -0.04494374, ..., -0.10995812,
          0.00572557,  0.09915379],
        [ 0.01873984,  0.04382846, -0.04304255, ..., -0.07801618,
         -0.0784068 , -0.00304188]], shape=(4, 384)),
 'documents': ["Rohit Sharma is the most successful captain in IPL history, leading Mumbai Indians to five titles. He's known for his calm demeanor and ability to play big innings under pressure.",
  'MS Dhoni, famously known as Captain Cool, has led Chennai Super Kings to multiple IPL titles. His finishing skills, wicketkeeping, and leadership are legendary.',
  'Jasprit Bumrah is